# Майнор "Интеллектуальный анализ данных"

# Курс "Прикладные задачи анализа данных"

## Лабораторная работа №2. RecSys

В рамках данной лабораторной работы вам предлагается решить задачу рекомендательной системы на основе следующих данных:

* **rating.csv** - рейтинги аниме по 10 бальной шкале
* **anime.csv** - описание items

Скачать данные можно [здесь](https://drive.google.com/drive/u/1/folders/1FarHUuqQq4tQSlERB9K8uEfZAZT9tQWv)

# Данные

Данные содержат информацию об предподчтениях 73,516 пользователей в 12,294 аниме. Каждый пользователь может добавить аниме и выставить рейтниг.

Anime.csv

* anime_id - идентификатор аниме (items)
* name - полное название аниме
* genre - категория/категории для аниме (разделены запятой).
* type - тип: movie, TV, OVA, etc.
* episodes - количество эпизодов (для экранизаций)
* rating - средний рейтинг по аниме.
* members - количество членов коммьюнити к аниме

Rating.csv

* user_id - идентификатор пользователя (users)
* anime_id - идентификатор аниме
* rating - рейтинг пользователя по аниме (-1 пользователь смотрел, но не оставил рейтинг).

### ``` Если вычеслительные ресурсы не позволяют решить задачу на всех данных, то необходимо это сделать на "сэмпле" данных (выбрав от 5% ids' для user и item). Этот же подход справедлив и для оценки работы алгоритомв```

## Необходимо сделать:

In [ ]:
import pandas as pd
import numpy  as np
import seaborn as sns
import random
import matplotlib.pyplot as plt

In [ ]:
def draw(data):
    sns_plot = sns.distplot(data, color="blue")
    fig = sns_plot.get_figure()

In [ ]:
rating = pd.read_csv("rating.csv")
anime = pd.read_csv("anime.csv")

In [ ]:
print(f"Anime Data size: {len(anime)}")
print(f"Rating Data size: {len(rating)}")

In [ ]:
print(f'User Id Unique {len(rating.user_id.unique())}')
print(f'Anime Id Unique {len(rating.anime_id.unique())}')

### Попробуем взять 5 процентов для ускорения наших действий

In [ ]:
short_user_ids = rating['user_id'].sample(int(len(rating.user_id.unique()) * 0.05))
short_anime_ids = rating['anime_id'].sample(int(len(rating.anime_id.unique()) * 0.05))
len(short_anime_ids)

In [ ]:
rating = rating[(rating['user_id'].isin(short_user_ids)) & (rating['anime_id'].isin(short_anime_ids))]
len(rating)

In [ ]:
print(len(rating.user_id.unique()))
print(len(rating.anime_id.unique()))

Безрейтинговые выбранные пользователи и аниме не вошли в нашу выборку

### Задание №1 : Exploratory analysis (2 балла):

* Распределение числа пользователей по количеству взаимодействий
* Разпределение числа "айтемов" по количеству взаимодействий
* Распределение числа рейтингов, средних рейтингов по пользователям, по "айтемам"
* и т.д. (бонус)

### Распределение числа пользователей по количеству взаимодействий

In [ ]:
draw_data = rating.user_id.value_counts().values
draw(draw_data)

### Разпределение числа "айтемов" по количеству взаимодействий

In [ ]:
draw_data = rating.anime_id.value_counts().values
draw(draw_data)

### Распределение числа рейтингов, средних рейтингов по пользователям, по "айтемам"

#### Число рейтингов по юзерам

In [ ]:
draw_data = rating.groupby('user_id').rating.nunique().values
draw(draw_data)

#### Число рейтингов по айтемам

In [ ]:
draw_data = rating.groupby('anime_id').rating.nunique().values
draw(draw_data)

#### Средний рейтинг аниме

In [ ]:
draw_data = rating[rating.rating != -1].groupby('anime_id').rating.mean().values
draw(draw_data)

### Задание №2 : Оценить разреженность данных по рейтингу (1/2 балла)

In [ ]:
users_id_index = { id: index for index, id in enumerate(rating.user_id.unique()) }
anime_id_index = { id: index for index, id in enumerate(rating.anime_id.unique())}
matrix = np.zeros((rating.user_id.unique().size, rating.anime_id.unique().size))
for ind, iterr in rating.iterrows():
    matrix[users_id_index[iterr.user_id], anime_id_index[iterr.anime_id]] = iterr.rating
matrix.shape

In [ ]:
sns.heatmap(matrix)

### Оценим разреженность

In [ ]:
denom = rating.user_id.unique().size * rating.anime_id.unique().size
print(f'Sparsity {(1.0 - ((np.count_nonzero(matrix) * 1.0) / denom)) * 100} %')

### Задание №3 : Разделить данные на тренировочные и валидационные (1/2 балла)

In [ ]:
matrix_train = matrix.copy()
test_val = []
for user_i in range(matrix_train.shape[0]):
    test_sample = (matrix_train[user_i] > 7).nonzero()[0]
    test_val.append(random.sample(list(test_sample), k=(5 if 5 <= len(test_sample) else len(test_sample))))
    for anime_i in test_sample:
        matrix_train[user_i][anime_i] = 0
test_val

### Задание №4 : Решить задачу на основе предложения всем юзерам наиболее популярных item (1 балл)

In [ ]:
popul = np.count_nonzero(matrix_train, axis=0)
rat = sorted(range(len(popul)), key=lambda k: popul[k])
rec_popular = [rating.anime_id.unique()[i] for i in rat]
print(rec_popular[:10])
print(rat[:10])

### Задание №5 : Решить задачу на основе коллоборативной фильтрации (2 балла)

* Реализовать один из методов коллоборативной фильтрации SVD, SVD++, ALS, ALS with implicit feedback

### SVD

In [ ]:
def argmax(a, k):
    return np.argpartition(a, -k)[-k:]

In [ ]:
from numpy.linalg import svd

p = 32
u, d, v = svd(matrix)
U = u[:, :p]
D = np.diag(d)[:p, :p]
V = v[:p, :]
svd_pred = U @ D @ V

In [ ]:
result = argmax(svd_pred[0], 5)
result

In [ ]:
anime.iloc[result]

In [ ]:
anime.iloc[result].describe()

### Задание №6 Решить задачу на основе контент-based подхода, выбрав один из методов решения (2 балла):

* На основание векторов с факторизационной матрицы
* При помощи "ембеддингов"  item'ов

#### Для наиболее понравишегося аниме найдем похожие на основании векторов с факторизационной матрицы

In [ ]:
recs = []
for user_i in range(matrix_train.shape[0]):
    best_index = np.where(matrix_train[user_i] == matrix_train[user_i].max())[0][0]
    recs.append(((V.T - V.T[best_index])**2).mean(axis=1).argsort()[1:1 + 5])
based_by_cont = np.array(recs)
based_by_cont

### Задание №7 Оценить работу алгоритмов выбрав одну из метрик, сделать вывод по результатам работы (2 балла):

### MAP@k


In [ ]:
def ap(actual, pred):
        score = 0.0
        hits = 0.0
        for i,p in enumerate(pred):
            if p in actual:    
                hits += 1.0
                score += hits / (i+1.0)
        if hits != 0:
            return score / hits 
        return 0

def mapka(actual, pred):
    scores = []
    for a,p in zip(actual, pred):
        if len(a) >= len(p):
            scores.append(ap(a, p))
        continue
    return np.mean(scores)

## Content 

In [ ]:
mapka(test_val, based_by_cont)

## Popular

In [ ]:
mapka(test_val, np.stack([rat[:5]] * len(test_val)))

## SVD

In [ ]:
mapka(tes_val, result)

## Выводы

1) SVD очень хорошо работает на выборках, которые имеют небольшой размер. Реккомендиции логически объяснимы и в какой-то степени схожи.

2) Content Based ничего практического не показала, так как основывается на количестве взаимодействий и результаты не воспроизводит

``` Бонус (1 балл) - дополнительные графики и характеристики в EDA и правильно сделанные выводы```

# ```Дедлайн - 08 июня 23:59```